In [ ]:
!pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.8 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=00ba72ded0c47b63ffa6c36ab2ebf6400f9d4706a1edcf92f9aad30db7699cdd
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
     QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from functools import partial
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit
from sklearn.base import clone
from ISLP.models import sklearn_sm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
import random
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
df = load_data('Boston')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  lstat    506 non-null    float64
 12  medv     506 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 51.5 KB


In [ ]:
#A
mu_hat = np.mean(df['medv'])
print(f'Estimated mean poblation: {mu_hat}')

#B
std = np.std(df['medv'])
n_samples = df['medv'].shape[0]

#C
standard_error_hat = std/np.sqrt(n_samples)
print(f"Standard error of the mean (SE_μ̂): {standard_error_hat:.2f}")


Estimated mean poblation: 22.532806324110677
Standard error of the mean (SE_μ̂): 0.41


In [ ]:
def boot_fn(data, indices):

    samples = data[indices]
    std = np.std(samples)
    n_samples = len(data[indices])
    standard_error_hat = std/np.sqrt(n_samples)
    return standard_error_hat


np.random.seed(2025)
estandard_errors = []
for i in range(1, 101):
    indices = np.random.choice(df.index, size=len(df), replace = True)
    se_hat = boot_fn(df['medv'].values, indices)
    estandard_errors.append(se_hat)

print(f'The standard error calculated before was: {standard_error_hat:.3f}')
print(f'The standard error hat is: {np.mean(estandard_errors):.3f}')



The standard error calculated before was: 0.408
The standard error hat is: 0.406


The standard error is almost equal, if we increment the number of iterations of the bootstrap we would get an even closer result.

In [ ]:
np.random.seed(2025)
estandard_errors = []
for i in range(1, 1000):
    indices = np.random.choice(df.index, size=len(df), replace = True)
    se_hat = boot_fn(df['medv'].values, indices)
    estandard_errors.append(se_hat)

print(f'The standard error calculated before was: {standard_error_hat:.3f}')
print(f'The standard error hat is: {np.mean(estandard_errors):.3f}')



The standard error calculated before was: 0.408
The standard error hat is: 0.407


In [ ]:
#8.d.1

def boot_fn(data, indices):
    samples = data[indices]
    return np.mean(samples)
mean = []
for _ in range(1,101):
    indices = np.random.choice(df.index, len(df), replace = True)
    mean_hat = boot_fn(df['medv'].values, indices)
    mean.append(mean_hat)
lower_bound = np.percentile(mean, 2.5)
upper_bound = np.percentile(mean, 97.5)
print(f'The boostrap Confidance interval with 95% lower limit is: {lower_bound:.4f} & the upper limit is: {upper_bound:.4f}')

The boostrap Confidance interval with 95% lower limit is: 21.8256 & the upper limit is: 23.4901


In [ ]:
from scipy.stats import t
sample_mean = np.mean(df['medv'])
sample_std = np.std(df['medv'])
n = len(df)
se_hat = sample_std/np.sqrt(n)

alpha = 0.05
t_critical = t.ppf(1 - alpha/2, df=n-1) # df as degrees of freedom

lower_limit = sample_mean - t_critical * se_hat
upper_limit = sample_mean + t_critical * se_hat

print(f'The boostrap Confidance interval with 95% lower limit is: {lower_bound:.4f} & the upper limit is: {upper_bound:.4f}')
print(f'The T test with 95% lower limit is: {lower_limit:.4f} & the upper limit is: {upper_limit:.4f}')


The boostrap Confidance interval with 95% lower limit is: 0.3699 & the upper limit is: 0.4359
The T test with 95% lower limit is: 21.7303 & the upper limit is: 23.3353


Using the bootstrap approach, we obtain a wider interval compared to the t-test. This is likely due to the bootstrap capturing more variability, as it does not assume normality and instead incorporates the full distributional characteristics of the data.

In [ ]:
#8E
median_hat = np.median(df['medv'])



#8F
sample_median = np.median(df['medv'])
bootstrap_medians = []
n_samples = len(df)
n_bootstraps = 1000

for _ in range(n_bootstraps):

    indices = np.random.choice(df.index, size=n_samples, replace=True)
    bootstrap_sample = df['medv'].iloc[indices]
    bootstrap_medians.append(np.median(bootstrap_sample))

ste_median = np.std(bootstrap_medians, ddof=1)

print(f'Median of the sample (mu_hat_median): {sample_median:.3f}')
print(f'Standard Error of the sample (bootstrap): {ste_median:.3f}')



Median of the sample (mu_hat_median): 21.200
Standard Error of the sample (bootstrap): 0.365
Tenth percentile (mu_hat_median ->0.1): 12.750


In [ ]:

#8G
percentile_10 = np.percentile(df['medv'], 10)


bootstrap_percentile = []
for _ in range(n_bootstraps):
    indices = np.random.choice(df.index, size=n_samples, replace=True)
    bootstrap_sample = df['medv'].iloc[indices]
    bootstrap_percentile.append(np.percentile(bootstrap_sample, 10))


ste_percentile_10 = np.std(bootstrap_percentile)


print(f"Tenth percentile (mu_hat_median ->0.1: {percentile_10:.3f}")
print(f"Standard Error of the Tenth percentile (bootstrap): {ste_percentile_10:.3f}")


Tenth percentile (mu_hat_median ->0.1: 12.750
Standard Error of the Tenth percentile (bootstrap): 0.481
